# The Big 4 sports' successfulness ratio
In this code snippet, I am going to compare the successfulness ratio of some clubs from "Big 4" sports, including NFL (footbal), MLB (baseball), NBA (basketball) and NHL (hockey), in the US. This ratio is defined as the number of wins over the number of wins plus the number of losses for each team. Then, I will utilize the population of each metropolitan and this ratio could to find their correlation with "pearsonr".

There are 5 datasets that are used for the following information:

1. Metropolitan regions and associated sports (wikipedia_data.html)
2. NFL (nfl.csv)
3. MLB (mlb.csv)
4. NBA (nba.csv)
5. NHL (nhl.csv)

## First Part
In this fraction, I calculate the successfulness correlation of NHL clubs with their region's population using their stats over 2018.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv('assets/nhl.csv')
cities = pd.read_html('assets/wikipedia_data.html')[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.set_index('Metropolitan area',inplace=True)

def nhl():
    # selecting date pertaining 2018
    df_team = nhl_df[nhl_df['year']==2018].iloc[:,[0,2,3]]
    # converting string to numeric
    df_team['W'] = pd.to_numeric(df_team['W'], errors='coerce')
    df_team['L'] = pd.to_numeric(df_team['L'], errors='coerce')
    # calculating win_ratio
    df_team['win_ratio']=df_team['W']/(df_team['W']+df_team['L'])

    # creating main dataframe
    df = pd.DataFrame(columns=['win_ratio','population','nhl_team'])
    df['population']=cities['Population (2016 est.)[8]']#.apply(float)
    df['nhl_team']= cities['NHL'].apply(lambda x:x.split("[")[0])
    without_team_name = df[(df['nhl_team']=='—')|(df['nhl_team']=='')].index
    df=df.drop(df[(df['nhl_team']=='—')|(df['nhl_team']=='')].index)


    # cleaning and separating teams column
    Metros = df['nhl_team'].index
    new_name = []
    for count,item in enumerate(df['nhl_team']):

        if " " in item:
            new_name.append([Metros[count],item])
        else:
            new_name.extend([[Metros[count],re.findall('[A-Z][^A-Z]*', item)]])


    # inserting new values to main df
    new_name = pd.DataFrame(new_name, columns=['Metropolitan area','team'])
    new_name.set_index('Metropolitan area',inplace=True)

    df['nhl_team'] = new_name


    # adding win_ratio values into main dataframe
    for count,teams in enumerate(df['nhl_team']):
        if isinstance(teams,str):
            location = df_team['team'].str.contains(teams)
            i = df_team[location].index
            df.iloc[count]['win_ratio']= df_team.iloc[i[0]]['win_ratio']
        else:
            summation = 0
            for team in range(len(teams)):
                location = df_team['team'].str.contains(teams[team])
                i = df_team[location].index
                summation += df_team.iloc[i[0]]['win_ratio']
            df.iloc[count]['win_ratio'] = summation/len(teams)
    return df.iloc[:,:-1]
nhl = nhl()
def nhl_correlation():
    

    df = nhl


#     print(df.iloc[:,[0,1]])
    population_by_region = df['population'].astype(np.float64) # pass in metropolitan area population from cities
    win_loss_by_region = df['win_ratio']    # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

# Run the function
print("The calculated correlation for NHL is {}".format(nhl_correlation()))

The calculated correlation for NHL is 0.01248616292120992


## Second Part
Now, I calculate the successfulness the correlation of NBA in 2018.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


nba_df=pd.read_csv('assets/nba.csv')
cities = pd.read_html('assets/wikipedia_data.html')[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.set_index('Metropolitan area',inplace=True)
def nba():
    
    sport = 'nba_team'
    # selecting date pertaining 2018
    df_team = nba_df[nba_df['year']==2018].iloc[:,[0,2,3]]
    df_team['W/L%'] = df_team['W/L%'].astype(np.float64)

    # creating main dataframe
    df = pd.DataFrame(columns=['win_ratio','population',sport])
    df['population']=cities['Population (2016 est.)[8]']
    df[sport]= cities['NBA'].apply(lambda x:x.split("[")[0])
    without_team_name = df[(df[sport]=='—')|(df[sport]=='')].index
    df=df.drop(df[(df[sport]=='—')|(df[sport]=='')].index)

    # cleaning and separating teams column
    Metros = df[sport].index
    new_name = []
    for count,item in enumerate(df[sport]):

        if " " in item:
            new_name.append([Metros[count],item])
        else:
    #         print(item)
            new_name.extend([[Metros[count],re.findall('[A-Z][^A-Z]*', item)]])

    # inserting a values to main df
    new_name = pd.DataFrame(new_name, columns=['Metropolitan area','team'])
    new_name.set_index('Metropolitan area',inplace=True)
    df[sport] = new_name


    # adding win_ratio values into main dataframe
    for count,teams in enumerate(df[sport]):
        if len(teams)==0:
            pass
        elif isinstance(teams,str):
            location = df_team['team'].str.contains(teams)
            i = df_team[location].index
            df.iloc[count]['win_ratio']= df_team.iloc[i[0]]['W/L%']
        else:
            summation = 0
            for team in range(len(teams)):
                location = df_team['team'].str.contains(teams[team])
                i = df_team[location].index
                summation += df_team.iloc[i[0]]['W/L%']
            average = summation/len(teams)
            df.iloc[count]['win_ratio'] = average
    return df.iloc[:,[0,1]]
nba = nba()
def nba_correlation():

    df = nba

    # passing in metropolitan area population from cities
    population_by_region = df['population'].astype(np.float64)
    win_loss_by_region = df['win_ratio']
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    mask=win_loss_by_region.isna()
    return stats.pearsonr(population_by_region[~mask], win_loss_by_region[~mask])[0]

# Run the function
print("The calculated correlation for NBA is {}".format(nba_correlation()))

The calculated correlation for NBA is -0.18861086968719679


## Third Part
The same process for MLB:

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# setting index
cities.set_index('Metropolitan area',inplace=True)  

def mlb():
    sport = 'mlb'
    # selecting date pertaining 2018
    df_team = mlb_df[mlb_df['year']==2018]#.iloc[:,[0,3]]
    df_team['W-L%'] = df_team['W-L%'].astype(np.float64)
    # creating main dataframe
    df = pd.DataFrame(columns=['win_ratio','population','mlb'])
    df['population']=cities['Population (2016 est.)[8]']
    df['mlb']= cities['MLB'].apply(lambda x:x.split("[")[0])


#     without_team_name = df[(df['mlb']=='—')|(df['mlb']=='')].index
    df=df.drop(df[(df['mlb']=='—')|(df['mlb']=='')].index)

    # cleaning and separating teams column
    pattern_1 = r"(\s)(?P<upper>[A-Z][^A-Z]*)"
    pattern_2 = r"[A-Z]{1}[^A-Z]*"
    Metros = df['mlb'].index
    new_name = []
    for count,item in enumerate(df['mlb']):
        for item1 in re.finditer(pattern_1, item):
            item = item.replace(item1.groupdict()['upper'],item1.groupdict()['upper'].lower())
        teams = []
        for item2 in re.finditer(pattern_2, item):
            if " " in item2[0]:
                string = list(item2[0])
                string[string.index(" ")+1] = string[string.index(" ")+1].upper()
                string = "".join(string)
                teams.append(string)
            else:
                teams.append(item2[0])
        new_name.append([Metros[count], teams])

    new_name = pd.DataFrame(new_name, columns=['Metropolitan area','team'])
    new_name.set_index('Metropolitan area',inplace=True)
    df['mlb'] = new_name

    # adding win_ratio values into main dataframe
    for count,teams in enumerate(df['mlb']):
        if len(teams)==0:
            pass
        elif isinstance(teams,str):
            location = df_team['team'].str.contains(teams)
            i = df_team[location].index
            df.iloc[count]['win_ratio']= df_team.iloc[i[0]]['W-L%']
        else:
            summation = 0
            for team in range(len(teams)):
                location = df_team['team'].str.contains(teams[team])
                i = df_team[location].index
                summation += df_team.iloc[i[0]]['W-L%']
            average = summation/len(teams)
            df.iloc[count]['win_ratio'] = average
    return df.iloc[:,[0,1]]
mlb = mlb()
def mlb_correlation():
    df = mlb

    # passing in metropolitan area population from cities    
    population_by_region = df['population'].astype(np.float64)
    win_loss_by_region = df['win_ratio']

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

# Run the function
print("The calculated correlation for MLB is {}".format(mlb_correlation()))

The calculated correlation for MLB is 0.15003737475409498


/home/bijan/anaconda3/envs/DM/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


## Fourth
That relation for NFL is calculated in the same way:

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl():
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # setting index
    cities.set_index('Metropolitan area',inplace=True)


    # selecting date pertaining 2018
    teams_stat_2018 = nfl_df[nfl_df['year']==2018].iloc[1:,[-3,-2]]
    df_team=[]
    df_win_ratio = []
    for ind in teams_stat_2018.index:
        try:
            df_win_ratio.append(float(teams_stat_2018['W-L%'].loc[ind]))
            df_team.append(teams_stat_2018['team'].loc[ind])#.astype(np.float64)
        except ValueError:
            pass

    df_team = pd.DataFrame({'team':df_team, 'W-L%':df_win_ratio})
    # creating main dataframe
    df = pd.DataFrame(columns=['win_ratio','population','nfl'])
    df['population']=cities['Population (2016 est.)[8]']
    df['nfl']= cities['NFL'].apply(lambda x:x.split("[")[0])
    without_team_name = df[(df['nfl']=='—')|(df['nfl']=='')].index
    df=df.drop(df[(df['nfl'].str.strip()=='—')|(df['nfl'].str.strip=='')].index)

    # cleaning and separating teams column
    pattern_1 = r"(\s)(?P<upper>[A-Z][^A-Z]*)"
    pattern_2 = r"[0-9A-Z]{1}[^A-Z]*"
    Metros = df['nfl'].index
    new_name = []
    for count,item in enumerate(df['nfl']):
        for item1 in re.finditer(pattern_1, item):
            item = item.replace(item1.groupdict()['upper'],item1.groupdict()['upper'].lower())
        teams = []
        for item2 in re.finditer(pattern_2, item):
            if " " in item2[0]:
                string = list(item2[0])
                string[string.index(" ")+1] = string[string.index(" ")+1].upper()
                string = "".join(string)
                teams.append(string)
            else:
                teams.append(item2[0])
        new_name.append([Metros[count], teams])

    new_name = pd.DataFrame(new_name, columns=['Metropolitan area','team'])
    new_name.set_index('Metropolitan area',inplace=True)
    df['nfl'] = new_name

    # adding win_ratio values into main dataframe
    for count,teams in enumerate(df['nfl']):
        if len(teams)==0:
            pass
        elif isinstance(teams,str):
            location = df_team['team'].str.contains(teams)
            i = df_team[location].index
            df.iloc[count]['win_ratio']= df_team.iloc[i[0]]['W-L%']
        else:
            summation = 0

            for team in range(len(teams)):
                location = df_team['team'].str.contains(teams[team])
                i = df_team[location].index
                summation += df_team.iloc[i[0]]['W-L%']

            average = summation/len(teams)
            df.iloc[count]['win_ratio'] = average
    df = df.dropna()
    return df.iloc[:,[0,1]]
nfl = nfl()
def nfl_correlation():

    df = nfl

    population_by_region = df['population'].astype(np.float64)
    win_loss_by_region = df['win_ratio'] 

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

print("The calculated correlation for MLB is {}".format(nfl_correlation()))

The calculated correlation for MLB is 0.004282141436393008


## Fifth
The relation between the population of a region and the successfulness of its clubs among their prospective clubs is scrutinied in this section. The ttest_rel from the scipy.stats module was used to evaluate this hypothesis. The rigions where a sports does not have an instance were excluded from the calculation of that place.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_ind


mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def merging_dfs(df1, df2):
    return pd.merge(df1,df2, how="inner", left_index=True, right_index=True).dropna()

def p_value_calc(df):
    ttest_ind,pval=stats.ttest_rel(df.iloc[:,0],df.iloc[:,1])
    return ttest_ind,pval

def sports_team_performance():


#     raise NotImplementedError()
    
#     Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
#     df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    sports_stats = pd.DataFrame({'NFL':nfl['win_ratio'], 'NBA':nba['win_ratio'],
                                'NHL':nhl['win_ratio'], 'MLB':mlb['win_ratio']})
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for sport1 in sports:
        for sport2 in sports:
            df = merging_dfs(sports_stats[sport1], sports_stats[sport2])
            p_values.loc[sport1, sport2] = p_value_calc(df)[1]
    
#     print(p_values)
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

print("The performance of  {}".format(sports_team_performance()))

          NFL       NBA       NHL       MLB
NFL       NaN  0.876115  0.030318  0.803459
NBA  0.876115       NaN  0.019022  0.880018
NHL  0.030318  0.019022       NaN  0.000703
MLB  0.803459  0.880018  0.000703       NaN
The performance of            NFL       NBA       NHL       MLB
NFL       NaN  0.876115  0.030318  0.803459
NBA  0.876115       NaN  0.019022  0.880018
NHL  0.030318  0.019022       NaN  0.000703
MLB  0.803459  0.880018  0.000703       NaN
